<a href="https://colab.research.google.com/github/giseldo/artigo-storypoint-deep-se-llm/blob/main/distilbert_uncase_story_point_quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U "optimum[onnxruntime]" onnxruntime>=1.1

In [ ]:
!pip install transformers optimum[onnxruntime] onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
from pathlib import Path
from transformers import AutoTokenizer
from optimum.exporters.onnx import main_export
from optimum.onnxruntime import ORTModelForSequenceClassification, ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig

model_id = "giseldo/distilbert_bert_uncased_finetuned_story_point"
out_dir  = Path("onnx"); out_dir.mkdir(parents=True, exist_ok=True)

# 1) exporta seu DistilBERT fine-tuned para ONNX
main_export(model_id, output=str(out_dir), task="text-classification")

# 2) carrega o modelo ONNX recém-gerado
ort_model  = ORTModelForSequenceClassification.from_pretrained(out_dir, file_name="model.onnx")
tokenizer  = AutoTokenizer.from_pretrained(model_id)

# 3) cria um quantizer e escolhe quantização dinâmica INT8 (AVX2 funciona em quase todo x86-64)
quantizer  = ORTQuantizer.from_pretrained(ort_model)
dqconfig   = AutoQuantizationConfig.avx2(is_static=False, per_channel=False)

# 4) aplica a quantização  ➜  salva em onnx/model_quantized.onnx
quantized_path = quantizer.quantize(save_dir=out_dir, quantization_config=dqconfig)
print("✅ Quantizado em:", quantized_path)


✅ Quantizado em: onnx
